<a href="https://colab.research.google.com/github/mlgancena/ge120/blob/main/GANCE%C3%91A_ME4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ME 4

1. output CSV files (RESULTS.csv) containing the Point, Northings, and Eastings
2. output text file (README.txt) containing sample REC, LEC, description and results csv descriptions
3. user may input any t1 northings or eastings
4. program should not break when error is encountered from uploading the file up to outputting the results
5. math, pandas and google.colab modules
6. program should free up temporary storage upon successful completion of solution and output of the results


In [ ]:
pip install pandas

In [ ]:
import pandas as pd

In [ ]:
import math

In [ ]:
def coord(line_data, prevnorth, preveast):
  '''
  Computes the Northings and Eastings of an observed point given the line distance and bearing

  Parameters
   line_data:   tuple , tuple of line name, distance and bearing
   prevnorth:   float , Northings of the previous point
   preveast:    float , Eastings of the previous point

  Return
  line_list:    list , list containing the following values:
      calcnorth:  float , calculated Northings of observed point
      calceast:   float , calculated Eastings of observed point
      lat:        float , calculated latitude of line
      dep:        float , calculated departure of line
  '''
  # dms to decimal degrees
  decimal_deg = math.radians(float(line_data[3]) + (float((line_data[4]))/60) + (float((line_data[5]))/3600))
  dist = float(line_data[1])

  if line_data[2] == 'N' and line_data[6] == 'E': # north east
    lat = dist * math.cos(decimal_deg)
    dep = dist * math.sin(decimal_deg)
  elif line_data[2] == 'N' and line_data[6] == 'W': # north west
      lat = dist * math.cos(decimal_deg)
      dep = dist * math.sin(decimal_deg) * -1
  elif line_data[2] == 'S' and line_data[6] == 'E': # south east
      lat = dist * math.cos(decimal_deg) * -1
      dep = dist * math.sin(decimal_deg)
  elif line_data[2] == 'S' and line_data[6] == 'W': # south west
      lat = dist * math.cos(decimal_deg) * -1
      dep = dist * math.sin(decimal_deg) * -1
  elif line_data[2] == 'None':
    lat = 0
    if line_data[6] == 'E': # due East
      dep = dist * math.sin(decimal_deg)
    elif line_data[6] == 'W': # due West
      dep = dist * math.sin(decimal_deg) * -1
  elif line_data[6] == 'None':
    dep = 0
    if line_data[2] == 'N': # due North
      lat = dist * math.cos(decimal_deg)
    elif line_data[2] == 'S': # due South
      lat = dist * math.cos(decimal_deg) * -1

  calceast = preveast + dep # calculated eastings and northings
  calcnorth = prevnorth + lat

  line_list = [calcnorth, calceast, lat, dep]
  return line_list

In [ ]:
def check_accuracy(sumdist, sumlat, sumdep):
  '''
  Computes the LEC, REC and accuracy standards of the traverse

  Parameters
   sumdist:   float , sum of all distances of traverse lines
   sumlat:    float , sum of all latitudes of traverse lines
   sumdep:    float , sum of all departures of traverse lines

  Return
  accu_list:  list , list containing the following values:
      lec:          float , linear error of closure of the traverse
      rec_denom:    float , denominator of the relative error of closure
      aacuracy:     string , specifies what control standards were met
  '''
  lec = round(((sumlat**2) + (sumdep**2))**(1/2),4) # lec at rec_denom calculations
  rec_denom = (math.floor((sumdist/lec)/1000))*1000

  # find what accuracy standards were satisfied
  if rec_denom >= 100000:
      accuracy = 'First Order Geodetic Control'
  elif rec_denom >= 50000:
      accuracy = 'Second Order Geodetic Control'
  elif rec_denom >= 20000:
      accuracy = 'Third Order Geodetic Control'
  elif rec_denom >= 10000:
      accuracy = 'Fourth Order Geodetic Control'
  else:
      accuracy = 'Does not satisfy geodetic control standards'

  accu_list = [lec, rec_denom, accuracy]
  return accu_list

In [ ]:
try:
  with open('ME4-2324-TRAVERSE.txt','r') as tdata: # with statements close and free up storage after running
    while True:
      try:
        coord_data = [float(input('TP1 Northings: ')), float(input('TP1 Eastings: '))]
      except ValueError: # non numbers are inputted
        print('Please input numbers! Try again\n')
      else:
        break

    columns = tdata.readline() # reads the first line
    data = tdata.readlines() # puts remaining lines into a list
    total_dist = 0 # assign variables for calculations and storage
    total_lat = 0
    total_dep = 0
    point_counter = 1 # for point labeling purposes
    point_list = []
    point_list.append('T'+ str(point_counter)) # adds 'T1' to point list
    north_list = [coord_data[0]] # add inputted 'T1' northings and eastings
    east_list = [coord_data[1]]

    for line in data: # for every remaining line in file
      strip_data = line.strip('\n') # remove \n at the end of each line, \n creates the rows
      dict_data = strip_data.split('\t') # data per line was separated by \t, \t creates the columnar look

      try:
        coord_data = coord(dict_data, coord_data[0], coord_data[1]) # function returns N, E, lat and dep
      except NameError: # NameError - either due to math or function names
        print('Run \'coord\' function AND \'import\' cells first then try again ')
        break

      total_dist = total_dist + float(dict_data[1]) # variables to be used for readme file and check_accuracy
      total_lat = total_lat + coord_data[2]
      total_dep = total_dep + coord_data[3]

      point_counter += 1
      point_list.append('T'+ str(point_counter)) # update and append the next point name
      north_list.append(coord_data[0]) # append the next northings and eastings
      east_list.append(coord_data[1])

    try:
      del point_list[-1] # last sighted point is the first point, replace it with 'T1'
      point_list.append(point_list[0])
    except: # IndexError - due to codes above this not working
      print('Make sure that ALL functions and imports have been executed')


    with open('README.txt','w+') as readme: # text file containing sample REC, LEC, description and results.csv descriptions
      try:
        accu_output = check_accuracy(total_dist, total_lat, total_dep) # function returns lec, rec denominator and accuracy
        list_output = ['LEC: ', str(accu_output[0]), '\nREC:  1  :  ', str(accu_output[1]),
                      '\nAccuracy: ', str(accu_output[2])]
        readme.writelines(list_output) # writes lines above in txt file
        list_desc = ['\n-----------------------------------------------------\n',
                      'The traverse has ', str(point_counter - 1), ' points \n',
                      'No corrections were made with T1 backsight used: ', str(north_list[0]), ' N ',
                      str(east_list[0]), ' E\n', 'The T1 coordinates found in the last row are the foresight to close the loop']
        readme.writelines(list_desc) # writes lines above in txt file

      except NameError: # NameError - either due to math, previous function or current function names
        print('Run \'check accuracy\' function first then try again')


except FileNotFoundError: # file is not uploaded to colab
  print('File is not found\nUpload traverse text file first then try again')

TP1 Northings: 1620574.565
TP1 Eastings: 506576.200


In [ ]:
try:
  traversedata = pd.DataFrame({'Point': point_list, 'Northings': north_list, 'Eastings': east_list})
except NameError: # pandas not working/imported
  print('Make sure that \'pandas\' have been installed and imported')

traversedata # output csv containing point name, northings and eastings

,Point,Northings,Eastings
0,T1,1620574.565,506576.200
1,T2,1620576.377,506573.814
2,T3,1620578.225,506572.433
3,T4,1620581.700,506570.974
4,T5,1620718.088,506571.676
5,T6,1620720.536,506580.657
6,T7,1620721.499,506585.477
7,T8,1620721.022,506589.840
8,T9,1620722.151,506594.437
9,T10,1620724.863,506598.564


In [ ]:
  traversedata.to_csv('RESULTS.csv') # creates csv file